# Model Fine Tuning
----
**Objective**: In this notebook, you will get to fine tune a transformer model on MRPC dataset. You will get to go through the whole process of transfer learning from loading the dataset, loading the model, compiling the model, training the model, and finally evaluating your results.

NOTE: Make sure to change the runtime from CPU to TPU or GPU for faster training

## Install Libraries
Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00


## Import Dataset
___
In this section we will use as an example the MRPC (Microsoft Research Paraphrase Corpus) dataset. The dataset consists of 5,801 pairs of sentences, with a label indicating if they are paraphrases or not (i.e., if both sentences mean the same thing).
This is one of the 10 datasets composing the GLUE benchmark, which is an academic benchmark that is used to measure the performance of ML models across 10 different text classification tasks.


**Question 1**: Use the "load_datasets" function to load the "mrpc" dataset from the "glue" benchmark.

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset('glue','mrpc')  # Add your solution here

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

## Pre-processing data

First, we need to tokenize our data. We will do so using the BERT tokenizer. We will use the AutoTokenizer with truncation.
This will truncate token by token, removing a token from the longest sequence in the pair if a pair of sequences (or a batch of pairs) is provided.

**Question 2:** Use the .map() function to apply the tokenization functionon the raw dataset. Hint: set the batched flag to True for faster processing.

In [6]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Then, we need to pad our data so that all input sequences have the same lengths.

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

Lets take a training example to see what our dataset looks like.

**Question 3:** Display an example from the training section of the dataset. Hint: The tokenized_datasets variable is a nested dictionary where the main keys are "train", "test", and "validate"

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [13]:
tokenized_datasets['train'][2]

{'sentence1': 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
 'sentence2': "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .",
 'label': 1,
 'idx': 2,
 'input_ids': [101,
  2027,
  2018,
  2405,
  2019,
  15147,
  2006,
  1996,
  4274,
  2006,
  2238,
  2184,
  1010,
  5378,
  1996,
  6636,
  2005,
  5096,
  1010,
  2002,
  2794,
  1012,
  102,
  2006,
  2238,
  2184,
  1010,
  1996,
  2911,
  1005,
  1055,
  5608,
  2018,
  2405,
  2019,
  15147,
  2006,
  1996,
  4274,
  1010,
  5378,
  1996,
  14792,
  2005,
  5096,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

Finally, we will prepare our training and validation datasets by transforming them into tensorflow datasets.

In [14]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Model Preperation
---
In this section, we will load the model and equip it with a classifier head. This model is already pre-trained and we will fine tune it with the dataset for paraphrasing classification that we prepared.

### Load Classifier Head

In [15]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Compile the model

We will compile the model using SparseCategoricalCrossentropyloss, which is when there are two or more label classes that are not one hot encded.

**Question 4:** Compile the model using the adam optimizer, SparseCategoricalCrossentropy loss (with from_logits flag set to True) and displaying the accuracy metric.

In [18]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf

loss = SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
history=model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Train the model

Now we will train our model using the processing training and validation sets.

**Question 5:** Fit the model on the training and validation sections that we prepared earlier.

In [19]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=5)

Epoch 1/5
459/459 [==============================] - 140s 193ms/step - loss: 0.5822 - accuracy: 0.7067 - val_loss: 0.4104 - val_accuracy: 0.8284
Epoch 2/5
459/459 [==============================] - 74s 161ms/step - loss: 0.4115 - accuracy: 0.8176 - val_loss: 0.4165 - val_accuracy: 0.8235
Epoch 3/5
459/459 [==============================] - 75s 164ms/step - loss: 0.2494 - accuracy: 0.9109 - val_loss: 0.3847 - val_accuracy: 0.8382
Epoch 4/5
459/459 [==============================] - 72s 158ms/step - loss: 0.1276 - accuracy: 0.9567 - val_loss: 0.5138 - val_accuracy: 0.8407
Epoch 5/5
459/459 [==============================] - 73s 160ms/step - loss: 0.0769 - accuracy: 0.9779 - val_loss: 0.4986 - val_accuracy: 0.8529


## Evaluating the model

Let's test how accurate the models are! In this section, we will predict on the testing dataset to check the model's performance when it comes to classifying paraphrased sentences on unseen data.

In [20]:
preds = model.predict(tf_test_dataset)["logits"]

216/216 [==============================] - 19s 62ms/step


In [21]:
import numpy as np
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(1725, 2) (1725,)


In [22]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets["test"]["label"])

{'accuracy': 0.8115942028985508, 'f1': 0.8630425621576066}